# Collinear spin and magnetic systems

In this example we consider iron in the BCC phase.
To show that this material is ferromagnetic we will model it once
allowing collinear spin polarization and once without
and compare the resulting SCF energies. In particular
the ground state can only be found if collinear spins are allowed.

First we setup BCC iron without spin polarization
using a single iron atom inside the unit cell.

In [1]:
using DFTK

a = 5.42352  # Bohr
lattice = a / 2 * [[-1  1  1];
                   [ 1 -1  1];
                   [ 1  1 -1]]
atoms     = [ElementPsp(:Fe, psp=load_psp("hgh/lda/Fe-q8.hgh"))]
positions = [zeros(3)];

To get the ground-state energy we use an LDA model and rather moderate
discretisation parameters.

In [2]:
kgrid = [3, 3, 3]  # k-point grid (Regular Monkhorst-Pack grid)
Ecut = 15          # kinetic energy cutoff in Hartree
model_nospin = model_LDA(lattice, atoms, positions, temperature=0.01)
basis_nospin = PlaneWaveBasis(model_nospin; kgrid, Ecut)

scfres_nospin = self_consistent_field(basis_nospin; tol=1e-6, mixing=KerkerDosMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.65001469330                   -0.48    5.2
  2   -16.65069637743       -3.17       -1.01    1.0
  3   -16.65080937472       -3.95       -2.32    1.8
  4   -16.65082421675       -4.83       -2.81    3.0
  5   -16.65082466335       -6.35       -3.28    2.0


In [3]:
scfres_nospin.energies

Energy breakdown (in Ha):
    Kinetic             15.9191204
    AtomicLocal         -5.0683687
    AtomicNonlocal      -5.2192736
    Ewald               -21.4723040
    PspCorrection       1.8758831 
    Hartree             0.7790614 
    Xc                  -3.4466559
    Entropy             -0.0182873

    total               -16.650824663352

Since we did not specify any initial magnetic moment on the iron atom,
DFTK will automatically assume that a calculation with only spin-paired
electrons should be performed. As a result the obtained ground state
features no spin-polarization.

Now we repeat the calculation, but give the iron atom an initial magnetic moment.
For specifying the magnetic moment pass the desired excess of spin-up over spin-down
electrons at each centre to the `Model` and the guess density functions.
In this case we seek the state with as many spin-parallel
$d$-electrons as possible. In our pseudopotential model the 8 valence
electrons are 2 pair of $s$-electrons, 1 pair of $d$-electrons
and 4 unpaired $d$-electrons giving a desired magnetic moment of `4` at the iron centre.
The structure (i.e. pair mapping and order) of the `magnetic_moments` array needs to agree
with the `atoms` array and `0` magnetic moments need to be specified as well.

In [4]:
magnetic_moments = [4];

!!! tip "Units of the magnetisation and magnetic moments in DFTK"
    Unlike all other quantities magnetisation and magnetic moments in DFTK
    are given in units of the Bohr magneton $μ_B$, which in atomic units has the
    value $\frac{1}{2}$. Since $μ_B$ is (roughly) the magnetic moment of
    a single electron the advantage is that one can directly think of these
    quantities as the excess of spin-up electrons or spin-up electron density.

We repeat the calculation using the same model as before. DFTK now detects
the non-zero moment and switches to a collinear calculation.

In [5]:
model = model_LDA(lattice, atoms, positions; magnetic_moments, temperature=0.01)
basis = PlaneWaveBasis(model; Ecut, kgrid)
ρ0 = guess_density(basis, magnetic_moments)
scfres = self_consistent_field(basis, tol=1e-6; ρ=ρ0, mixing=KerkerDosMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Magnet   Diag
---   ---------------   ---------   ---------   ------   ----
  1   -16.66157930672                   -0.51    2.618    5.0
  2   -16.66812772649       -2.18       -1.09    2.445    1.4
  3   -16.66905583805       -3.03       -2.06    2.338    2.2
  4   -16.66910025608       -4.35       -2.69    2.303    2.0
  5   -16.66910308351       -5.55       -2.91    2.296    2.5
  6   -16.66910444201       -5.87       -3.54    2.286    2.1
  7   -16.66910446162       -7.71       -3.89    2.286    3.1


In [6]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             16.2944690
    AtomicLocal         -5.2225729
    AtomicNonlocal      -5.4098875
    Ewald               -21.4723040
    PspCorrection       1.8758831 
    Hartree             0.8191505 
    Xc                  -3.5406821
    Entropy             -0.0131606

    total               -16.669104461616

!!! note "Model and magnetic moments"
    DFTK does not store the `magnetic_moments` inside the `Model`, but only uses them
    to determine the lattice symmetries. This step was taken to keep `Model`
    (which contains the physical model) independent of the details of the numerical details
    such as the initial guess for the spin density.

In direct comparison we notice the first, spin-paired calculation to be
a little higher in energy

In [7]:
println("No magnetization: ", scfres_nospin.energies.total)
println("Magnetic case:    ", scfres.energies.total)
println("Difference:       ", scfres.energies.total - scfres_nospin.energies.total);

No magnetization: -16.650824663351823
Magnetic case:    -16.669104461615543
Difference:       -0.018279798263719726


Notice that with the small cutoffs we use to generate the online
documentation the calculation is far from converged.
With more realistic parameters a larger energy difference of about
0.1 Hartree is obtained.

The spin polarization in the magnetic case is visible if we
consider the occupation of the spin-up and spin-down Kohn-Sham orbitals.
Especially for the $d$-orbitals these differ rather drastically.
For example for the first $k$-point:

In [8]:
iup   = 1
idown = iup + length(scfres.basis.kpoints) ÷ 2
@show scfres.occupation[iup][1:7]
@show scfres.occupation[idown][1:7];

(scfres.occupation[iup])[1:7] = [1.0, 0.9999987819425997, 0.9999987819425993, 0.9999987819425991, 0.9582262035814257, 0.9582262035798416, 1.127051203892608e-29]
(scfres.occupation[idown])[1:7] = [1.0, 0.8436597277759427, 0.8436597277753437, 0.8436597277715393, 8.118456176084531e-6, 8.118456174736153e-6, 1.585070792759098e-32]


Similarly the eigenvalues differ

In [9]:
@show scfres.eigenvalues[iup][1:7]
@show scfres.eigenvalues[idown][1:7];

(scfres.eigenvalues[iup])[1:7] = [-0.06936128603468089, 0.35688587945820094, 0.3568858794623456, 0.3568858794634284, 0.4617402538313609, 0.46174025383175665, 1.1596220301986437]
(scfres.eigenvalues[idown])[1:7] = [-0.031260731781106985, 0.47621125607723913, 0.47621125607728454, 0.476211256077573, 0.610282024203384, 0.6102820242050448, 1.2252893389613893]


!!! note "$k$-points in collinear calculations"
    For collinear calculations the `kpoints` field of the `PlaneWaveBasis` object contains
    each $k$-point coordinate twice, once associated with spin-up and once with down-down.
    The list first contains all spin-up $k$-points and then all spin-down $k$-points,
    such that `iup` and `idown` index the same $k$-point, but differing spins.

We can observe the spin-polarization by looking at the density of states (DOS)
around the Fermi level, where the spin-up and spin-down DOS differ.

In [10]:
using Plots
plot_dos(scfres)

Similarly the band structure shows clear differences between both spin components.

Computing bands along kpath:
       Γ -> H -> N -> Γ -> P -> H  and  P -> N
Diagonalising Hamiltonian kblocks: 100%|████████████████| Time: 0:00:01


In [11]:
using Unitful
using UnitfulAtomic
plot_bandstructure(scfres; kline_density=6)